<!-- 
Author: Wenyu Ouyang
Date: 2022-09-06 08:51:01
LastEditTime: 2022-09-11 15:20:44
LastEditors: Wenyu Ouyang
Description: Get ERA5-LAND data for buffers of sites and transform them to CAMELS format
FilePath: \CatchmentForcings\catchmentforcings\ecmwf4basins\download_era5land_gee.ipynb
Copyright (c) 2021-2022 Wenyu Ouyang. All rights reserved.
-->
# Download ERA5-LAND for some basins from GEE

Get ERA5-LAND data for basins and transform them to CAMELS format

We try two ways to implement:

1. We firstly choose data in one day, then we perform the basin-mean operation for each hour and then calculate the daily mean/sum value. Then we loop over all days in the time period.
2. We firstly average/sum the data in one day, then we perform the basin-mean operation for each day.

You can try any of them, but the results format are different. Then you have to use different scripts to transform them to CAMELS format.

## time-first average

In [1]:
'''
Author: Wenyu Ouyang
Date: 2023-11-08 16:20:21
LastEditTime: 2023-11-08 16:21:55
LastEditors: Wenyu Ouyang
Description: 
FilePath: \CatchmentForcings\catchmentforcings\ecmwf4basins\download_era5land4basins_gee.ipynb
Copyright (c) 2023-2024 Wenyu Ouyang. All rights reserved.
'''
import ee, eemont, geemap
import pandas as pd
import numpy as np

In [2]:
ee.Authenticate()


Successfully saved authorization token.


In [2]:
ee.Initialize()

*** Earth Engine *** Share your feedback by taking our Annual Developer Satisfaction Survey: https://google.qualtrics.com/jfe/form/SV_doiqkQG3NJ1t8IS?source=API


some functions:

In [4]:
# reduce hourly featurecollection to daily
def nestedMap(dayCol, feaCol, selectors):
    def mapDayCol(oneDay):
        chosen = feaCol.filter(ee.Filter.eq('date', oneDay))
        chosen_reduce = chosen.reduceColumns(**{
            "selectors": selectors,
            "reducer": ee.Reducer.mean().repeat(10).group(
                **{'groupField': 0,
                   'groupName': "ID"})})
        chosen_reduce_dict_lst = chosen_reduce.values()[0]
        data = ee.List(chosen_reduce_dict_lst).map(
            lambda d: ee.Dictionary(
                {
                    "gage_id": ee.Dictionary(d)["ID"], 
                    selectors[1]: ee.List(ee.Dictionary(d)["mean"])[0], 
                    selectors[2]:ee.List(ee.Dictionary(d)["mean"])[1],
                    selectors[3]:ee.List(ee.Dictionary(d)["mean"])[2],
                    selectors[4]:ee.List(ee.Dictionary(d)["mean"])[3],
                    selectors[5]:ee.List(ee.Dictionary(d)["mean"])[4],
                    selectors[6]:ee.List(ee.Dictionary(d)["mean"])[5],
                    # the followings should be sum, here we directly use daily mean * 24 h
                    selectors[7]:ee.Number(ee.List(ee.Dictionary(d)["mean"])[6]) * ee.Number(24),
                    selectors[8]:ee.Number(ee.List(ee.Dictionary(d)["mean"])[7]) * ee.Number(24),
                    selectors[9]:ee.Number(ee.List(ee.Dictionary(d)["mean"])[8]) * ee.Number(24),
                    selectors[10]:ee.Number(ee.List(ee.Dictionary(d)["mean"])[9]) * ee.Number(24),
                }
            )
        )
        csv_feature_lst = data.map(
            lambda f: ee.Feature(None, f).set(
                {
                    "date": oneDay
                }
            )
        )
        csv_feat_col = ee.FeatureCollection(csv_feature_lst)
        return csv_feat_col

    # https://developers.google.com/earth-engine/apidocs/ee-featurecollection-merge
    # If many collections need to be merged, consider placing them all in a collection and using FeatureCollection.flatten() instead.
    return dayCol.map(mapDayCol).flatten()

In [7]:
# main function
def era5land4basins(year_num, feature_file, file_name="chinachosenbasins", save_gdrive_dir="ERA5LAND"):
    fc = ee.FeatureCollection(feature_file)
    era5land_bands = [
        "dewpoint_temperature_2m",
        "temperature_2m",
        "skin_temperature",
        "u_component_of_wind_10m",
        "v_component_of_wind_10m",
        "surface_pressure",
        "surface_net_solar_radiation_hourly",
        "potential_evaporation_hourly",
        "total_evaporation_hourly",
        "total_precipitation_hourly"
    ]
    year = ee.Number(year_num)
    month = ee.Number(1)
    day = ee.Number(1)
    start_date = ee.Date.fromYMD(year, month, day)
    # end_date = start_date.advance(2, "day")
    end_date = start_date.advance(1, "year")
    days_num = end_date.difference(start_date, "day")
    # count day from zero, and ee.List.sequence is a closed interval
    days = ee.List.sequence(ee.Number(0), days_num.add(-1))
    dates = days.map(lambda t: start_date.advance(t, "day").format("YYYYMMdd"))
    era5_land = (ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")
                 .filterBounds(fc)
                 .filterDate(start_date,end_date)
                 .index(era5land_bands))
    ts = era5_land.getTimeSeriesByRegions(reducer = ee.Reducer.mean(),
                                          collection = fc,
                                          bands = era5land_bands,
                                          dateFormat = 'YYYYMMdd',
                                          scale = 11132)
    selectors = ["ID"] + era5land_bands    
    reduce_to_daily = nestedMap(dates, ts, selectors)
    save_reduce_to_daily = ee.FeatureCollection(reduce_to_daily).flatten()
    # export to google drive
    save_file = f"era5land_{file_name}_mean_" + str(year_num)
    geemap.ee_export_vector_to_drive(
        save_reduce_to_daily,
        description=save_file,
        folder=save_gdrive_dir,
        fileFormat="csv",
        selectors=[
            "gage_id",
            "date",
        ] + era5land_bands,
    )

In [8]:
# numpy int is int32 which is not our need
year_nums = [1980 + i for i in range(10)]
for year_num in year_nums:
    era5land4basins(year_num,feature_file="projects/ee-owen/assets/filtered_china_basins")

Exporting era5land_chinachosenbasins_mean_1980... Please check the Task Manager from the JavaScript Code Editor.
Exporting era5land_chinachosenbasins_mean_1981... Please check the Task Manager from the JavaScript Code Editor.
Exporting era5land_chinachosenbasins_mean_1982... Please check the Task Manager from the JavaScript Code Editor.
Exporting era5land_chinachosenbasins_mean_1983... Please check the Task Manager from the JavaScript Code Editor.
Exporting era5land_chinachosenbasins_mean_1984... Please check the Task Manager from the JavaScript Code Editor.
Exporting era5land_chinachosenbasins_mean_1985... Please check the Task Manager from the JavaScript Code Editor.
Exporting era5land_chinachosenbasins_mean_1986... Please check the Task Manager from the JavaScript Code Editor.
Exporting era5land_chinachosenbasins_mean_1987... Please check the Task Manager from the JavaScript Code Editor.
Exporting era5land_chinachosenbasins_mean_1988... Please check the Task Manager from the JavaScr

## basin-first average

In [3]:
import ee
import geemap

In [4]:
# Image Collection
era5_land = ee.ImageCollection("ECMWF/ERA5_LAND/HOURLY")

# Visualization parameters
temperatureVis = {
    'bands': ['temperature_2m'],
    'min': 250,
    'max': 320,
    'palette': [
        "#000080", "#0000D9", "#4000FF", "#8000FF", "#0080FF", "#00FFFF",
        "#00FF80", "#80FF00", "#DAFF00", "#FFFF00", "#FFF500", "#FFDA00",
        "#FFB000", "#FFA400", "#FF4F00", "#FF2500", "#FF0A00", "#FF00FF"
    ]
}

# Feature Collections
china_basins = ee.FeatureCollection("projects/ee-owen/assets/filtered_china_basins")

# Time range
year_start = 1980
year_end = 1981

# Collection Names
cb_name = "chinachosenbasins"

In [5]:
def nested_mapped_reducer_forcing(
    feat_col, img_col, scale_num, basinid_name="ID", date_name="day_of_all_years"
):
    def map_reducer_over_imgcol(feat):
        def img_reducer(img):
            vals = img.reduceRegion(
                reducer=ee.Reducer.mean(), geometry=feat.geometry(), scale=scale_num
            )
            return ee.Feature(None, vals).set(
                {
                    "time_start": img.get(date_name),
                    "gage_id": feat.get(basinid_name),
                }
            )

        return img_col.map(img_reducer)

    return feat_col.map(map_reducer_over_imgcol).flatten()

In [16]:
# Define the calculateAvg function
def calculateAvg(year_num, region_name, image_source, region_shape, time_zone):
    year = ee.Number(year_num)
    month = ee.Number(1)
    day = ee.Number(1)
    start_date = ee.Date.fromYMD(year, month, day).advance(time_zone, "hour")
    end_date = start_date.advance(2, "day")
    # end_date = start_date.advance(1, "year")
    print(end_date.getInfo())
    days_num = end_date.difference(start_date, "day")
    print("number of days", days_num.getInfo())
    days = ee.List.sequence(0, days_num.subtract(1))
    print(days.getInfo())

    # Get ImageCollection and filter, choose two days for test
    era5_land_2d = image_source.filter(ee.Filter.date(start_date, end_date))
    print(era5_land_2d.limit(10).getInfo())

    # Forcing variables that will be calculated for its average
    avg_forcings = era5_land_2d.select(
        [
            "dewpoint_temperature_2m",
            "temperature_2m",
            "skin_temperature",
            "soil_temperature_level_1",
            "soil_temperature_level_2",
            "soil_temperature_level_3",
            "soil_temperature_level_4",
            "lake_bottom_temperature",
            "lake_ice_depth",
            "lake_ice_temperature",
            "lake_mix_layer_depth",
            "lake_shape_factor",
            "lake_total_layer_temperature",
            "snow_albedo",
            "snow_cover",
            "snow_density",
            "snow_depth",
            "snow_depth_water_equivalent",
            "temperature_of_snow_layer",
            "skin_reservoir_content",
            "volumetric_soil_water_layer_1",
            "volumetric_soil_water_layer_2",
            "volumetric_soil_water_layer_3",
            "volumetric_soil_water_layer_4",
            "forecast_albedo",
            "u_component_of_wind_10m",
            "v_component_of_wind_10m",
            "surface_pressure",
            "leaf_area_index_high_vegetation",
            "leaf_area_index_low_vegetation",
        ]
    )
    print(avg_forcings.limit(10).getInfo())

    # Forcing variables that will be calculated for its sum
    sum_forcings = era5_land_2d.select(
        [
            "snowfall_hourly",
            "snowmelt_hourly",
            "surface_latent_heat_flux_hourly",
            "surface_net_solar_radiation_hourly",
            "surface_net_thermal_radiation_hourly",
            "surface_sensible_heat_flux_hourly",
            "surface_solar_radiation_downwards_hourly",
            "surface_thermal_radiation_downwards_hourly",
            "evaporation_from_bare_soil_hourly",
            "evaporation_from_open_water_surfaces_excluding_oceans_hourly",
            "evaporation_from_the_top_of_canopy_hourly",
            "evaporation_from_vegetation_transpiration_hourly",
            "potential_evaporation_hourly",
            "runoff_hourly",
            "snow_evaporation_hourly",
            "sub_surface_runoff_hourly",
            "surface_runoff_hourly",
            "total_evaporation_hourly",
            "total_precipitation_hourly",
        ]
    )
    print(sum_forcings.limit(10).getInfo())

    # Show temperature, just for test
    temperature = era5_land_2d.select(["temperature_2m"])
    # Add layer to an interactive map (If you are in a Jupyter environment)
    # Map.addLayer(temperature, temperatureVis, 'Air temperature [K] at 2m height')
    print(temperature.limit(10).getInfo())

    # Calculate the average values for each day
    avg_days = ee.ImageCollection(
        days.map(
            lambda day: avg_forcings.filter(
                ee.Filter.date(
                    start_date.advance(day, "day"),
                    start_date.advance(ee.Number(day).add(1), "day"),
                )
            )
            .reduce(ee.Reducer.mean())
            .set({"day_of_all_years": start_date.advance(day, "day")})
        )
    )
    sum_days = ee.ImageCollection(
        days.map(
            lambda day: sum_forcings.filter(
                ee.Filter.date(
                    start_date.advance(day, "day"),
                    start_date.advance(ee.Number(day).add(1), "day"),
                )
            )
            .reduce(ee.Reducer.sum())
            .set({"day_of_all_years": start_date.advance(day, "day")})
        )
    )

    avg_day_regions = nested_mapped_reducer_forcing(region_shape, avg_days, scale_num=11132)

    avg_day_regions_4sum = nested_mapped_reducer_forcing(region_shape, sum_days, scale_num=11132)

    # Export avg_day_regions to google drive using geemap
    # https://geemap.org/common/#geemap.common.ee_export_vector_to_drive
    geemap.ee_export_vector_to_drive(
        avg_day_regions,
        description=f"era5land_{region_name}_avg_mean_{year_num}",
        folder="ERA5LAND",
        fileFormat="csv",  # or 'GeoJSON', 'KML', 'KMZ' if needed
        selectors=[
            "gage_id",
            "time_start",
            "dewpoint_temperature_2m_mean",
            "temperature_2m_mean",
            "skin_temperature_mean",
            "soil_temperature_level_1_mean",
            "soil_temperature_level_2_mean",
            "soil_temperature_level_3_mean",
            "soil_temperature_level_4_mean",
            "lake_bottom_temperature_mean",
            "lake_ice_depth_mean",
            "lake_ice_temperature_mean",
            "lake_mix_layer_depth_mean",
            "lake_shape_factor_mean",
            "lake_total_layer_temperature_mean",
            "snow_albedo_mean",
            "snow_cover_mean",
            "snow_density_mean",
            "snow_depth_mean",
            "snow_depth_water_equivalent_mean",
            "temperature_of_snow_layer_mean",
            "skin_reservoir_content_mean",
            "volumetric_soil_water_layer_1_mean",
            "volumetric_soil_water_layer_2_mean",
            "volumetric_soil_water_layer_3_mean",
            "volumetric_soil_water_layer_4_mean",
            "forecast_albedo_mean",
            "u_component_of_wind_10m_mean",
            "v_component_of_wind_10m_mean",
            "surface_pressure_mean",
            "leaf_area_index_high_vegetation_mean",
            "leaf_area_index_low_vegetation_mean",
        ],
    )

    # Export avg_day_regions_4sum to google drive using geemap
    geemap.ee_export_vector_to_drive(
        avg_day_regions_4sum,
        description=f"era5land_{region_name}_sum_mean_{year_num}",
        folder="ERA5LAND",
        fileFormat="csv",  # or 'GeoJSON', 'KML', 'KMZ' if needed
        selectors=[
            "gage_id",
            "time_start",
            "snowfall_hourly_sum",
            "snowmelt_hourly_sum",
            "surface_latent_heat_flux_hourly_sum",
            "surface_net_solar_radiation_hourly_sum",
            "surface_net_thermal_radiation_hourly_sum",
            "surface_sensible_heat_flux_hourly_sum",
            "surface_solar_radiation_downwards_hourly_sum",
            "surface_thermal_radiation_downwards_hourly_sum",
            "evaporation_from_bare_soil_hourly_sum",
            "evaporation_from_open_water_surfaces_excluding_oceans_hourly_sum",
            "evaporation_from_the_top_of_canopy_hourly_sum",
            "evaporation_from_vegetation_transpiration_hourly_sum",
            "potential_evaporation_hourly_sum",
            "runoff_hourly_sum",
            "snow_evaporation_hourly_sum",
            "sub_surface_runoff_hourly_sum",
            "surface_runoff_hourly_sum",
            "total_evaporation_hourly_sum",
            "total_precipitation_hourly_sum",
        ],
    )

In [17]:
# Example of using the function for a range of years
# for "Asia/Hong_Kong" the time zone offset is -8, for example, beijing time is 0 then utc-0 is 16 of the last day
for year_num in range(year_start, year_end):
    calculateAvg(year_num, cb_name, era5_land, china_basins, -8)  # UTC time zone offset is 0

{'type': 'Date', 'value': 315676800000}
number of days 2
[0, 1]
{'type': 'ImageCollection', 'bands': [], 'id': 'ECMWF/ERA5_LAND/HOURLY', 'version': 1699493470175480.0, 'properties': {'type_name': 'ImageCollection', 'keywords': ['cds', 'climate', 'copernicus', 'ecmwf', 'era5-land', 'evaporation', 'heat', 'lakes', 'precipitation', 'pressure', 'radiation', 'reanalysis', 'runoff', 'snow', 'soil_water', 'temperature', 'vegetation', 'wind'], 'visualization_1_bands': 'total_precipitation', 'visualization_1_max': '0.1', 'description': '<p>ERA5-Land is a reanalysis dataset providing a consistent view of the evolution of land variables\nover several decades at an enhanced resolution compared to ERA5. ERA5-Land has been produced by\nreplaying the land component of the ECMWF ERA5 climate reanalysis. Reanalysis combines model\ndata with observations from across the world into a globally complete and consistent dataset\nusing the laws of physics. Reanalysis produces data that goes several decades ba

## download and format-transform

After getting dataset from GEE to google drive, download them from google drive to local computer and unzip the files.

In [1]:
!unzip ~/code/HydroDataDownloader/drive-download-20220626T013744Z-003.zip -d ~/code/HydroDataDownloader/ERA5

Archive:  /home/ouyangwenyu/code/HydroDataDownloader/drive-download-20220626T013744Z-003.zip
  inflating: era5_daily_sample_globalpaper_site3221_1985.csv  
  inflating: era5_daily_sample_globalpaper_site3221_1986.csv  


In [2]:
!unzip ~/code/HydroDataDownloader/drive-download-20220626T013744Z-002.zip -d ~/code/HydroDataDownloader/ERA5

Archive:  /home/ouyangwenyu/code/HydroDataDownloader/drive-download-20220626T013744Z-002.zip
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2003.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2008.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2010.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2014.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_1991.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2001.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2005.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2015.csv  
  inflating: /home/

In [3]:
!unzip ~/code/HydroDataDownloader/drive-download-20220626T013744Z-001.zip -d ~/code/HydroDataDownloader/ERA5

Archive:  /home/ouyangwenyu/code/HydroDataDownloader/drive-download-20220626T013744Z-001.zip
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2020.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2016.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_1997.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2009.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_1989.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_1996.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_2013.csv  
  inflating: /home/ouyangwenyu/code/HydroDataDownloader/ERA5/era5_daily_sample_globalpaper_site3221_1994.csv  
  inflating: /home/

In [1]:
era5_we_download = [
    # GEE下载数据的时候不小心处理错了，搞成 new_site_i 了，实际上是 new_site_id
    "new_site_i",
    "date",
    "tmean",
    "tmin",
    "tmax",
    "tdew",
    "prcp",
    "sp",
    "mslp",
    "windu",
    "windv",
]
camels_format_index = [
    "Year",
    "Mnth",
    "Day",
    "Hr",
    "tmean(K)",
    "tmin(K)",
    "tmax(K)",
    "tdew(K)",
    "prcp(m)",
    "sp(Pa)",
    "mslp(Pa)",
    "windu(m/s)",
    "windv(m/s)",
]

In [2]:
import os

era5_dir = os.path.join(os.getcwd(), "ERA5")
output_dir = os.path.join(os.getcwd(), "ERA5_globalpaper_sites3221")

In [3]:
import pandas as pd

gage_dict = pd.read_csv(os.path.join(os.getcwd(), "globalpaper_site3221.csv"), sep=",")
gage_id_key = "new_site_id"

In [6]:
import numpy as np

years_all = np.arange(1980, 2021)

In [7]:
import fnmatch

from tqdm.notebook import tqdm, trange

for year in tqdm(years_all):
    for f_name in os.listdir(era5_dir):
        if fnmatch.fnmatch(
            f_name, "era5_daily_sample_globalpaper_site3221_" + str(year) + ".csv"
        ):
            data_file = os.path.join(era5_dir, f_name)
            break
    data_temp = pd.read_csv(data_file, sep=",")
    for i_basin in tqdm(range(len(gage_dict[gage_id_key]))):
        # name csv
        basin_data = data_temp[
            data_temp[era5_we_download[0]].values == gage_dict[gage_id_key][i_basin]
        ]
        if basin_data.shape[0] == 0:
            raise ArithmeticError("chosen basins' number is zero")
        # get Year,Month,Day,Hour info
        csv_date = pd.to_datetime(basin_data[era5_we_download[1]])
        # the hour is set to 12, as 12 is the average hour of a day
        year_month_day_hour = pd.DataFrame(
            [[dt.year, dt.month, dt.day, 12] for dt in csv_date],
            columns=camels_format_index[0:4],
        )
        data_df = pd.DataFrame(
            basin_data.iloc[:, 2:].values, columns=camels_format_index[4:]
        )
        # concat
        new_data_df = pd.concat([year_month_day_hour, data_df], axis=1)
        # output the result
        if not os.path.isdir(output_dir):
            os.makedirs(output_dir)
        output_file = os.path.join(
            output_dir, gage_dict[gage_id_key][i_basin] + "_site_era5_forcing.txt"
        )
        # print("output forcing data of", gage_dict[gage_id_key][i_basin], "year", str(year))
        if os.path.isfile(output_file):
            data_old = pd.read_csv(output_file, sep=" ")
            years = np.unique(data_old[camels_format_index[0]].values)
            if year in years:
                continue
            else:
                os.remove(output_file)
                new_data_df = pd.concat([data_old, new_data_df]).sort_values(
                    by=camels_format_index[0:3]
                )
        new_data_df.to_csv(
            output_file, header=True, index=False, sep=" ", float_format="%.6f"
        )

  0%|          | 0/36 [00:00<?, ?it/s]

  0%|          | 0/3221 [00:00<?, ?it/s]

  0%|          | 0/3221 [00:00<?, ?it/s]

  0%|          | 0/3221 [00:00<?, ?it/s]

  0%|          | 0/3221 [00:00<?, ?it/s]

...

  0%|          | 0/3221 [00:00<?, ?it/s]

  0%|          | 0/3221 [00:00<?, ?it/s]

  0%|          | 0/3221 [00:00<?, ?it/s]

  0%|          | 0/3221 [00:00<?, ?it/s]

  0%|          | 0/3221 [00:00<?, ?it/s]

最后压缩下文件，方便下载下来

In [8]:
!zip -r ~/code/HydroDataDownloader/ERA5_globalpaper_sites3221.zip ~/code/HydroDataDownloader/ERA5_globalpaper_sites3221

  adding: home/ouyangwenyu/code/HydroDataDownloader/ERA5_globalpaper_sites3221/ (stored 0%)
  adding: home/ouyangwenyu/code/HydroDataDownloader/ERA5_globalpaper_sites3221/g2432_site_era5_forcing.txt (deflated 62%)
  adding: home/ouyangwenyu/code/HydroDataDownloader/ERA5_globalpaper_sites3221/g327_site_era5_forcing.txt (deflated 63%)
  adding: home/ouyangwenyu/code/HydroDataDownloader/ERA5_globalpaper_sites3221/g1777_site_era5_forcing.txt (deflated 63%)
  adding: home/ouyangwenyu/code/HydroDataDownloader/ERA5_globalpaper_sites3221/g85_site_era5_forcing.txt (deflated 62%)
  adding: home/ouyangwenyu/code/HydroDataDownloader/ERA5_globalpaper_sites3221/g497_site_era5_forcing.txt (deflated 62%)
  adding: home/ouyangwenyu/code/HydroDataDownloader/ERA5_globalpaper_sites3221/g2710_site_era5_forcing.txt (deflated 62%)
  adding: home/ouyangwenyu/code/HydroDataDownloader/ERA5_globalpaper_sites3221/g2101_site_era5_forcing.txt (deflated 63%)
  adding: home/ouyangwenyu/code/HydroDataDownloader/ERA5_g